<a href="https://colab.research.google.com/github/mkjubran/Fundamentals-of-AI-and-Machine-Learning/blob/main/NEURAL_NETWORKS_USING_PyTorch_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fitting and Evaluating Convolutional Neural Network using PyTorch


In this notebook, we will demonstrate how to fit and evaluate a ConvolutionalNeural Network (CNN) Model. We will work on the Medical MNIST dataset from Kaggle (https://www.kaggle.com/code/stpeteishii/medical-mnist-6-classify-torchvision-cnn/data).

This notebok is based on the code in the notebook https://www.kaggle.com/code/stpeteishii/medical-mnist-6-classify-torchvision-cnn/notebook

we will start by importing packages and modules necessary for the execution of this notebook.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, models 
from torchvision.utils import make_grid

from sklearn.model_selection import train_test_split

# Obtain Data

**Clone the dataset Repository**

We used a subset of medical images in the original dataset for this notebook. This dataset can be cloned from the GitHub repository https://github.com/mkjubran/MedicalImagingData.git as below

In [ ]:
!rm -rf /content/MedicalImagingData
!rm -rf ./AIData
!git clone https://github.com/mkjubran/MedicalImagingData.git

# Prepare Data

Each class of images is placed in a separate folder in the path '/content/MedicalImagingData/Dataset1/'. The name of the folder is the label of the images. To get the labels, we will read the list of folder names.

In [ ]:
dir0='/content/MedicalImagingData/Dataset1/'
names0=os.listdir(dir0)
print(sorted(names0))


The dataset has about 6000 images. It is important to apply data augmentation techniques to train the CNN model. We will use torchvision.transforms module to alter the data before training the CNN model.

In [ ]:
train_transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor()
])

''',
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
'''

We will use the torchvision.datasets utility classes to build our own datasets.

In [ ]:
dataset=datasets.ImageFolder(root=dir0,transform=train_transform)
display(dataset)

To test the created dataset

In [ ]:
class_names=dataset.classes
print(class_names)
print(len(class_names))

Now, we will split the data into training and testing datasets.

In [ ]:
train_indices, test_indices = train_test_split(list(range(len(dataset.targets))), test_size=0.2, stratify=dataset.targets)
train_data = torch.utils.data.Subset(dataset, train_indices)
test_data = torch.utils.data.Subset(dataset, test_indices)

print('Size of the dataset = {}'.format(len(dataset.imgs)))
print('Size of the training dataset = {} ({}%)'.format(len(train_data.indices), 100*len(train_data.indices)/len(dataset.imgs)))
print('Size of the testing dataset = {} ({}%)'.format(len(test_data.indices), 100*len(test_data.indices)/len(dataset.imgs)))

Next,we will use Dataloader to feed in the data to the model during training. we wil. use batch size of 100 images.

In [ ]:
batch_size=100
train_loader=DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_data,batch_size=batch_size)

To view the images and the labeles in the train dataloader

In [ ]:
#check the 1st batch
for images, labels in train_loader:
    break

print('Label:', labels.numpy()) # here, labels are given as single numbers.
print('Class:', *np.array([class_names[i] for i in labels]))

im = make_grid(images,nrow=10)

plt.figure(figsize=(10,10))
plt.imshow(np.transpose(im.numpy(),(1,2,0)))
plt.show()

We need to set the deive to run the deep learning. We would llike to have GPUs but the model can run with low speed on CPUs.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Using {} device'.format(device))

To build the CNN model using Conved layers.

In [ ]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #Ho=floor[((Hin+2P-d*(K-1))/S)+1], Ho=floor[((224+2*0-1*(3-1))/1)+1]=222, default values; padding=0, dilation=1
        self.conv1=nn.Conv2d(in_channels=3,out_channels=6,kernel_size=3,stride=1)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1)
        self.fc1=nn.Linear(in_features=16*54*54,out_features=120) 
        self.fc2=nn.Linear(in_features=120,out_features=84)
        self.fc3=nn.Linear(in_features=84,out_features=20)
        self.fc4=nn.Linear(in_features=20,out_features=len(class_names))
        
    def forward(self,X):
        #print(X.shape)  #torch.Size([10, 3, 224, 224]) #which means batch size and image size.
        X=F.relu(self.conv1(X))
        #print(X.shape) #[100, 6, 222, 222]
        X=F.max_pool2d(X,2,2)
        #print(X.shape) #[100, 6, 111, 111]
        X=F.relu(self.conv2(X))
        #print(X.shape) #[100, 16, 109, 109]
        X=F.max_pool2d(X,2,2)
        #print(X.shape)  #[10, 16, 54, 54]
        X=X.view(-1,16*54*54)
        #print(X.shape)  #[10, 46656]        
        X=F.relu(self.fc1(X))
        #print(X.shape)  #[10, 120] 
        X=F.relu(self.fc2(X))
        #print(X.shape)  #[10, 84]
        X=F.relu(self.fc3(X))
        #print(X.shape)  #[10, 20]
        X=self.fc4(X)
        #print(X.shape)  #[10, 6]
        return F.log_softmax(X,dim=1)

model = ConvolutionalNetwork().to(device)
print(model)

After defining the model, we need to set the loss function and the optimizer.

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

Training function including the steps of forward pass and backpropagation

In [ ]:
def train(dataloader, model, loss_fn, optimizer, epoch, trn_corr):
    batch_size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        batch+=1
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        y_pred = model(X)
        loss = criterion(y_pred, y)

        predicted=torch.max(y_pred.data,1)[1]
        batch_corr=(predicted==y).sum()
        trn_corr+=batch_corr

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 5 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Training: epoch: {epoch} loss: {loss} batch: {batch} accuracy: {trn_corr.item()*100/(current):7.3f}%")

    loss, current = loss.item(), batch * len(X)
    print(f"Training: epoch: {epoch} loss: {loss} batch: {batch} accuracy: {trn_corr.item()*100/(current):7.3f}%")
    return trn_corr, loss

Testing function to evaluate the model

In [ ]:
def test(dataloader, model, loss_fn, epoch, tst_corr):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
          batch+=1
          X, y = X.to(device), y.to(device)

          # Compute prediction error
          y_pred=model(X)
          loss=criterion(y_pred,y)

          predicted=torch.max(y_pred.data,1)[1]
          btach_corr=(predicted==y).sum()
          tst_corr+=btach_corr

    loss, current = loss.item(), batch * len(X)
    print(f"Testing: epoch: {epoch} loss: {loss} accuracy: {tst_corr.item()*100/(current):7.3f}%")

    return tst_corr, loss

Now, we will train and evaluate the model for 4 epochs while watch the training and testing accuracy.

In [ ]:
import time
start_time=time.time()
train_losses=[]
test_losses=[]
train_correct=[]
test_correct=[]
epochs=4

for i in range(epochs):
    trn_corr=0
    tst_corr=0
    
    trn_corr, loss = train(train_loader, model, criterion, optimizer, i, trn_corr)
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    tst_corr, loss = test(test_loader, model, criterion, i, tst_corr)
    test_losses.append(loss)
    test_correct.append(tst_corr)
    print(f'-----------------------------------\n')  

print(f'\nDuration: {time.time() - start_time:.0f} seconds') 

#Saving the Models

We wil use PyTorch to save the model

In [ ]:
torch.save(model.state_dict(), "CNN_model.pth")
print("Saved PyTorch Model State to CNN_model.pth")

# Predict New Values Using Models

Let us use the model to predict the class and compare it with the ground truth

not ready yet

In [ ]:
#for batch, (X, Y) in enumerate(test_loader):
samples_toshow=10
batch=0
for X, Y in test_loader:
        batch+=1
        X, y = X.to(device), y.to(device)
        x = X[0].to(device)
        y = Y[0].to(device)
        
        with torch.no_grad():
          y_pred = model(x)
          predicted=torch.max(y_pred.data,1)[1].item()
          
          print('Sample ', batch)
          print('True Label:', y.numpy()) # here, labels are given as single numbers.
          print('Predicted Label:', predicted) # here, labels are given as single numbers.

          print('True Class:', *np.array([class_names[y]]))
          print('Predicted Class:', *np.array([class_names[predicted]]))

          im = make_grid(x,nrow=10)
          plt.figure(figsize=(4,4))
          plt.imshow(np.transpose(im.numpy(),(1,2,0)))

          plt.show()
          time.sleep(2)

        if batch == samples_toshow:
          break